In [2]:
from brian2 import *
from brian2tools import *
import numpy as np
import os
from simulation.utils import connectrowcol, connectsub, getsubgrid

# Equation for the neuron model
eqs = '''
dv/dt = (I_d + I - v)/tau : 1
I = A*cos(2*pi*f*t)+ 0.5 : 1
I_d : 1
'''

# Function to run the simulation
def run_simulation(seedi, exc, inh, save_path):
    start_scope()
    np.random.seed(seedi)

    # Initialize the neuron group
    n = 81
    G1 = NeuronGroup(n, eqs, threshold='v>1', reset='v = 0', method='rk4')

    sudoku_grid = [
                    [2, 7, 5, 6, 3, 4, 1, 9, 8],
                    [3, 6, 4, 8, 1, 9, 2, 5, 7],
                    [1, 8, 9, 7, 2, 5, 3, 4, 6],
                    [9, 1, 8, 2, 5, 7, 6, 3, 4],
                    [5, 2, 7, 3, 4, 6, 8, 1, 9],
                    [4, 3, 6, 1, 9, 8, 7, 2, 5],
                    [6, 4, 3, 5, 7, 2, 9, 8, 1],
                    [7, 5, 2, 9, 8, 1, 4, 6, 3],
                    [8, 9, 1, 4, 6, 3, 5, 7, 2]
                    ]
    groupinit1 = [num for row in sudoku_grid for num in row]
   
    groupinit = zeros(81)
    for ii in range(n):
        if ii >= 0:
            a = groupinit1[ii]/10
            groupinit[ii] = a
    #groupinit = np.random.uniform(0.1, 0.11, n)
    #groupinit = [0.4979, 0.7255, 0.6580, 0.7010, 0.5484, 0.6299, 0.7285, 0.7851, 0.7635, 0.5484, 0.7010, 0.6253, 0.7635, 0.7243, 0.7869, 0.4979, 0.6580, 0.7255, 0.7285, 0.7635, 0.7851, 0.7255, 0.4979, 0.6580, 0.5484, 0.6299, 0.7010, 0.7851, 0.7243, 0.7680, 0.4979, 0.6580, 0.7310, 0.7067, 0.5484, 0.6253, 0.6580, 0.5055, 0.7255, 0.5562, 0.6253, 0.7010, 0.7635, 0.7243, 0.7869, 0.6299, 0.5484, 0.7010, 0.7243, 0.7869, 0.7635, 0.7255, 0.4979, 0.6580, 0.7010, 0.6299, 0.5484, 0.6643, 0.7255, 0.5055, 0.7851, 0.7680, 0.7243, 0.7255, 0.6580, 0.4979, 0.7869, 0.7635, 0.7243, 0.6253, 0.7010, 0.5484, 0.7635, 0.7869, 0.7243, 0.6253, 0.7010, 0.5484, 0.6580, 0.7255, 0.4979]
    G1.v = groupinit
    
    for ii in range(n):
        if ii >= 0:
            a = groupinit1[ii]/100
            groupinit[ii] = 0.61  #+ a + np.random.uniform(-0.1, 0.1)
    G1.I_d = groupinit
    
    sudo = G1.i[:].reshape(9, 9)
    inhibconnect(sudo)

    matrix = np.zeros((n, n))
    exc_matrix = 1 - matrix

    si, ti = matrix.nonzero()
    se, te = exc_matrix.nonzero()

    # Inhibitory synapses
    I1 = Synapses(G1, G1, on_pre='''v = v - inh ''')

    # Excitatory synapses
    E1 = Synapses(G1, G1, on_pre='''v = v + exc ''')

    I1.connect(i=si, j=ti, p=1)
    E1.connect(i=se, j=te)

    M1 = StateMonitor(G1, 'v', record=True)
    Sp1 = SpikeMonitor(G1)
    rate_mon = PopulationRateMonitor(G1)

    run(30000*ms)

    final_membrane_potentials = M1.v[:, -1]
    neuron_ids = np.arange(81)

    # Save spike train data
    spike_train_filename = os.path.join(save_path, f"spike_train_seedi_{seedi}_exc_{exc}_inh_{inh}.txt")
    with open(spike_train_filename, "w") as f:
        for neuron_id in range(81):
            spike_times = Sp1.t[Sp1.i == neuron_id] / ms
            spike_times_str = " ".join([str(st) for st in spike_times])
            f.write(f"{spike_times_str}\n")

    return neuron_ids, final_membrane_potentials, rate_mon


ModuleNotFoundError: No module named 'simulation'